In [2]:
import numpy as onp
from jax import random
import jax.numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import fwins_env as env
import memory
import agent
import pickle

In [75]:
load_model = True
model_name = "parameters_tsteps_4e5_mem_1_5e5_no_minmax"
used_loss = agent.zero_sum_loss
lr = 1e-3
preprocessing='distinguishPlayers'
minmax_training = 2
minmax_playing = 2

g = env.FourWins(preprocessing=preprocessing, trivial=False)

state_shape = g.preprocessedState(1)[0].shape

ag = agent.FeedforwardAgent(state_shape, [(4,3,16),(3,2,64), (100,)],
        (g.state.shape[0],), nonlinearity = ['relu','relu','relu','id'],
        used_loss = used_loss, model_name = model_name, learning_rate=lr,
        action_noise=0.01, gamma=0.98)

#     Restore variables from disk.
vs = np.load(model_name+'.npy', allow_pickle=True)
ag.variables = [[np.array(a) for a in b] for b in vs]
print("succesfully loaded model "+ ag.model_name+" from file!")

layer shape: (4, 3, 16)
conv
relu
layer shape: (3, 2, 64)
conv
relu
layer shape: (100,)
flatten to go from conv to dense
dense
relu
layer shape: (7,)
dense
shapes: [(7, 6, 2), (4, 3, 16), (3, 2, 64), (100,), (7,)]
succesfully loaded model parameters_tsteps_4e5_mem_1_5e5_no_minmax from file!


In [125]:
import copy
#rest of the stuff is just for testing the AI in 1v1 games
def get_ai_move(game, player, noise = 0., think_ahead = 0):
    s = game.preprocessedState(player)
    aiMove = None
    if think_ahead:
        aiMove = ag.think_ahead(game, player, think_ahead)[np.newaxis,:]
    else:
        aiMove = ag.prediction(s)
    a = onp.argmax(aiMove + onp.random.normal(size = aiMove.shape)*noise)
    #print(aiMove, a)
    return a, aiMove


def testgame(game = None, player_1_begins=False, ai_noise=0., thinking = 0):
    if game == None:
        game = env.FourWins(preprocessing=preprocessing)
    game_movements = []
    # performs a testgame user vs agent
    print("Playing against trained agent!")
    val = 0
    while (True):
        a = None
        if player_1_begins:
            a,aiMove = get_ai_move(game, 1, ai_noise, thinking)
        else:
            a,aiMove = get_ai_move(game, -1, ai_noise, thinking)
        
        val = game.play(1, a)

        #saving move
        player = "+" if val ==-1 else "o"
        game_movements.append((a,aiMove,player,copy.deepcopy(game)))

        if val == -1:
            print("Player o played illegal move and loses")
            break
        if val == 1:
            game.show()
            print("Player o wins!")
            break
        if not player_1_begins:
            a,aiMove = get_ai_move(game, -1, ai_noise, thinking)
        else:
            a,aiMove = get_ai_move(game, 1, ai_noise, thinking)
        
        val = game.play(-1, a)
        
        #saving move
        player = "o" if val ==-1 else "+"
        game_movements.append((a,aiMove,player,copy.deepcopy(game)))
        if val == -1:
            print("Player + played illegal move and loses")
            break
        if val == 1:
            game.show()
            print("Player + wins!")
            break
    return game_movements

In [126]:
player_1_begins = True      
game_moves = testgame(player_1_begins=player_1_begins,thinking=minmax_playing)

Playing against trained agent!
_________________
| o   + + + o + |
| +   o o o + o |
| o   + + + o o |
| +   + o o + + |
| o + o + + o o |
| + o + o o + o |
-----------------
Player + wins!


In [127]:
for move,game in enumerate(game_moves):
    print(move)
    print(game[0])
    print(game[1])
    print(game[2])
    game[3].show()

0
4
[[ 0.00412467 -0.00322278  0.00621822  0.00537459  0.00814317  0.00113609
   0.00576778]]
o
_________________
|               |
|               |
|               |
|               |
|               |
|         o     |
-----------------
1
4
[[0.00901567 0.00235267 0.01189629 0.00957212 0.01751784 0.00677923
  0.01042156]]
+
_________________
|               |
|               |
|               |
|               |
|         +     |
|         o     |
-----------------
2
4
[[0.00962771 0.0070736  0.01425811 0.01415101 0.0222833  0.01417417
  0.00957784]]
o
_________________
|               |
|               |
|               |
|         o     |
|         +     |
|         o     |
-----------------
3
4
[[0.0233096  0.01773863 0.01994105 0.02858104 0.02896379 0.02309743
  0.02056996]]
+
_________________
|               |
|               |
|         +     |
|         o     |
|         +     |
|         o     |
-----------------
4
3
[[0.02792832 0.01596936 0.02896875 0.03766249 0.03156213 